In [1]:
%%writefile mapper.py

import sys
import re


def eprint(*args, **kwargs):
    print(*args, file=sys.stderr, **kwargs)

for line in sys.stdin:
    try:
        article_id, text = line.strip().split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        eprint("reporter:counter:Wiki stats,Total words,%d" % 1)
        print("%s\t%d" % (word.lower(), 1))

Writing mapper.py


In [2]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

Writing reducer.py


In [3]:
%%writefile -a reducer.py

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print("%s\t%d" % (current_key, word_sum))
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print("%s\t%d" % (current_key, word_sum))

Appending to reducer.py


In [2]:
%%writefile mapper_1.py

import sys

for line in sys.stdin:
    try:
        word, count = line.strip().split('\t', 1)
        count = int(count)
        print("%d\t%s" % (count, word))
    except ValueError as e:
        continue

Writing mapper_1.py


In [3]:
%%writefile reducer_1.py

import sys

def eprint(*args, **kwargs):
    print(*args, file=sys.stderr, **kwargs)

for line in sys.stdin:
    try:
        count, word = line.strip().split('\t', 1)
        count = int(count)
        print("%s\t%d" % (word, count))
    except ValueError as e:
        continue

Writing reducer_1.py


In [4]:
! hdfs dfs -ls /data/

Found 2 items
drwxrwxrwx   - jovyan supergroup          0 2020-10-05 14:51 /data/twitter
drwxrwxrwx   - jovyan supergroup          0 2020-10-05 14:51 /data/wiki


In [9]:
%%bash

OUT_DIR="resultNonSorted"
OUT_DIR1="resultSorted"
NUM_REDUCERS=8
NUM_REDUCERS1=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null
hdfs dfs -rm -r -skipTrash ${OUT_DIR1} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Streaming Word Count" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python3 mapper.py" \
    -combiner "python3 reducer.py" \
    -reducer "python3 reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.reduces=${NUM_REDUCERS1} \
    -D mapred.jab.name="Sorting wordCount" \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D stream.map.output.field.separator="\t" \
    -D stream.num.map.output.key.fields=2 \
    -D mapreduce.map.output.key.field.separator="\t" \
    -D mapreduce.partition.keycomparator.options=-k1,1nr \
    -files mapper_1.py,reducer_1.py \
    -mapper 'python3 mapper_1.py' \
    -reducer 'python3 reducer_1.py' \
    -input ${OUT_DIR} \
    -output ${OUT_DIR1} > /dev/null

hdfs dfs -cat ${OUT_DIR1}/part-00000 | sed -n '7p;8q'

is	126420


rm: `resultNonSorted': No such file or directory
21/11/04 18:37:51 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/11/04 18:37:51 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/11/04 18:37:51 INFO mapred.FileInputFormat: Total input files to process : 1
21/11/04 18:37:51 INFO mapreduce.JobSubmitter: number of splits:2
21/11/04 18:37:51 INFO Configuration.deprecation: yarn.resourcemanager.system-metrics-publisher.enabled is deprecated. Instead, use yarn.system-metrics-publisher.enabled
21/11/04 18:37:51 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1636041870180_0010
21/11/04 18:37:52 INFO conf.Configuration: resource-types.xml not found
21/11/04 18:37:52 INFO resource.ResourceUtils: Unable to find 'resource-types.xml'.
21/11/04 18:37:52 INFO resource.ResourceUtils: Adding resource type - name = memory-mb, units = Mi, type = COUNTABLE
21/11/04 18:37:52 INFO resource.ResourceUtils: Adding resource type - name = vcores, units = 